In [1]:
import contextlib
import io

f = io.StringIO()
with contextlib.redirect_stdout(f):
    import pandas as pd
    import numpy as np
    import cvxpy as cp
    import matplotlib.pyplot as plt

    import warnings
    warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
df = pd.read_csv("data/100x100.csv")
df_2 = pd.read_csv("data/100x100_2.csv")

In [3]:
def baseline_model(_lambda,df):


    data_matrix = df.pivot(index='u_id', columns='a_id', values='score').fillna(0)
    delta = df[['u_id', 'a_id', 'score']]
    avg_rating = df['score'].mean()
    num_users = data_matrix.shape[0]
    num_anime = data_matrix.shape[1]

    b_u=cp.Variable(num_users)
    b_i=cp.Variable(num_anime)

    user_id_to_index = {} #helper function
    for idx in range(len(data_matrix.index)):
        u_id = data_matrix.index[idx]
        user_id_to_index[u_id] = idx
    
    anime_id_to_index = {} #helper function
    for idx in range(len(data_matrix.columns)):
        a_id = data_matrix.columns[idx]
        anime_id_to_index[a_id] = idx
        
    R_ui=[]
    for ui in range(len(delta)):
        u = user_id_to_index[int(delta.at[ui,'u_id'])]
        i = anime_id_to_index[int(delta.at[ui,'a_id'])]
        R_ui.append((avg_rating + b_u[u] + b_i[i])) #R_ui=avg_rating+b_u[u]+b_i[i]

    training_error=[]
    for ui in range(len(delta)):
        training_error.append((R_ui[ui]-int(delta.at[ui,'score']))**2) #(R_ui-A_ui)^2

    training_error_sum=cp.sum(training_error) #sum((R_ui-A_ui)^2)

    regularized_u=cp.sum_squares(b_u) #sum(b_u^2)
    regularized_i=cp.sum_squares(b_i) #sum(b_i^2)

    obj=cp.Minimize(training_error_sum+ _lambda*(regularized_u+regularized_i))
    prob=cp.Problem(obj)
    prob.solve()
    print("status:", prob.status)
    
    R = np.zeros((num_users, num_anime))
    for u in range(num_users):
        for i in range(num_anime):
            R[u, i] = avg_rating + b_u.value[u] + b_i.value[i]
    
    return np.matrix(R)

In [4]:
R = baseline_model(10000, df)

status: optimal
